# GonmAD analysis

In [1]:
from pathlib import Path
import os
import pandas

In [5]:
_folder_path = Path() / "Raw_data"
_raw_files = list(os.listdir(_folder_path))
print(_raw_files)

['gnomAD_v3.1.1_ENSG00000011677_2021_08_25_17_49_37_GABRA3.csv', 'gnomAD_v3.1.1_ENSG00000022355_2021_08_25_17_45_26_GABRA1.csv', 'gnomAD_v3.1.1_ENSG00000080644_2021_09_06_14_22_08_CHRNA3.csv', 'gnomAD_v3.1.1_ENSG00000094755_2021_08_25_17_53_59_GABRP.csv', 'gnomAD_v3.1.1_ENSG00000101204_2021_09_06_14_25_11_CHRNA4.csv', 'gnomAD_v3.1.1_ENSG00000101958_2021_08_26_22_07_30_GLRA2.csv', 'gnomAD_v3.1.1_ENSG00000102287_2021_08_25_17_51_00_GABRE.csv', 'gnomAD_v3.1.1_ENSG00000109158_2021_08_25_17_49_50_GABRA4.csv', 'gnomAD_v3.1.1_ENSG00000109738_2021_08_26_22_08_17_GLRB.csv', 'gnomAD_v3.1.1_ENSG00000111886_2021_08_25_17_54_36_GABRR2.csv', 'gnomAD_v3.1.1_ENSG00000113327_2021_08_25_17_52_13_GABRG2.csv', 'gnomAD_v3.1.1_ENSG00000120903_2021_08_26_22_10_13_CHRNA2.csv', 'gnomAD_v3.1.1_ENSG00000120903_2021_09_06_14_23_00_CHRNA2.csv', 'gnomAD_v3.1.1_ENSG00000138435_2021_08_26_22_09_48_CHRNA1.csv', 'gnomAD_v3.1.1_ENSG00000138435_2021_09_06_14_24_13_CHRNA1.csv', 'gnomAD_v3.1.1_ENSG00000145451_2021_08_26_22

## Importing the files
We import the file as pandas DataFrame and add a column in which we track the gene of interest.

In [9]:
data = []
for file in _raw_files:
    data.append(pandas.read_csv(_folder_path / file))
    data[-1]["Gene"] = file.split(".")[-2].split("_")[-1]

In [10]:
data[0]

,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,HGVS Consequence,Protein Consequence,...,Hemizygote Count Ashkenazi Jewish,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Gene
0,X,152168157,NaN,G,A,gnomAD Genomes,NaN,PASS,c.*71C>T,NaN,...,0,0,30604,0,0,0,53112,0,0,GABRA3
1,X,152168161,rs1384790433,C,T,gnomAD Genomes,NaN,PASS,c.*67G>A,NaN,...,0,1,30550,0,0,2,53058,0,0,GABRA3
2,X,152168162,rs1386272230,G,A,gnomAD Genomes,NaN,PASS,c.*66C>T,NaN,...,0,0,30614,0,0,3,53127,0,2,GABRA3
3,X,152168165,rs1603193519,G,A,gnomAD Genomes,NaN,PASS,c.*63C>T,NaN,...,0,1,30665,0,0,0,53180,0,0,GABRA3
4,X,152168167,rs1453252241,T,C,gnomAD Genomes,NaN,PASS,c.*61A>G,NaN,...,0,1,30551,0,0,0,53035,0,0,GABRA3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,X,152364593,rs1462667747,A,G,gnomAD Genomes,NaN,PASS,c.-23T>C,NaN,...,0,2,30588,0,1,0,53123,0,0,GABRA3
341,X,152364614,rs200503915,T,G,gnomAD Genomes,NaN,PASS,c.-26-18A>C,NaN,...,0,1,30551,0,0,8,53051,0,4,GABRA3
342,X,152364618,rs1380023644,T,A,gnomAD Genomes,NaN,PASS,c.-26-22A>T,NaN,...,0,0,30462,0,0,1,52933,0,0,GABRA3
343,X,152364628,rs1219359555,G,A,gnomAD Genomes,NaN,PASS,c.-26-32C>T,NaN,...,0,1,30672,0,0,0,53088,0,0,GABRA3


In [15]:
print(*list(data[0].columns), sep="\n")

Chromosome
Position
rsIDs
Reference
Alternate
Source
Filters - exomes
Filters - genomes
HGVS Consequence
Protein Consequence
Transcript Consequence
VEP Annotation
ClinVar Clinical Significance
ClinVar Variation ID
Flags
Allele Count
Allele Number
Allele Frequency
Homozygote Count
Hemizygote Count
Allele Count Other
Allele Number Other
Homozygote Count Other
Hemizygote Count Other
Allele Count Latino/Admixed American
Allele Number Latino/Admixed American
Homozygote Count Latino/Admixed American
Hemizygote Count Latino/Admixed American
Allele Count European (Finnish)
Allele Number European (Finnish)
Homozygote Count European (Finnish)
Hemizygote Count European (Finnish)
Allele Count Amish
Allele Number Amish
Homozygote Count Amish
Hemizygote Count Amish
Allele Count East Asian
Allele Number East Asian
Homozygote Count East Asian
Hemizygote Count East Asian
Allele Count Middle Eastern
Allele Number Middle Eastern
Homozygote Count Middle Eastern
Hemizygote Count Middle Eastern
Allele Count

In [17]:
print(*[len(x.columns) for x in data], sep="\n")

61
61
62
61
62
62
61
61
62
61
61
62
62
62
62
62
61
61
62
61
62
61
61
61
61
61
61
62
61
61
61
61
61
61
62
61
61
62
62
62
62
62
62
62
62
62
62


Some columns are present only in a subset of files. We look at one of those below.

In [18]:
set(data[2].columns) - set(data[0].columns)

{'Transcript'}

In [19]:
data[2]["Transcript"]

0      ENST00000348639.7
1      ENST00000348639.7
2      ENST00000348639.7
3      ENST00000348639.7
4      ENST00000348639.7
             ...        
598    ENST00000326828.6
599    ENST00000326828.6
600    ENST00000326828.6
601    ENST00000326828.6
602    ENST00000326828.6
Name: Transcript, Length: 603, dtype: object

We will keep them, and introduce NaN
## Joining the datasets

In [21]:
dataset = pandas.concat(data, axis=0, join="outer", ignore_index=True)
print(dataset)

      Chromosome     Position         rsIDs Reference Alternate  \
0              X  152168157.0           NaN         G         A   
1              X  152168161.0  rs1384790433         C         T   
2              X  152168162.0  rs1386272230         G         A   
3              X  152168165.0  rs1603193519         G         A   
4              X  152168167.0  rs1453252241         T         C   
...          ...          ...           ...       ...       ...   
28733          2  232545764.0  rs1329683219         A         G   
28734          2  232545766.0   rs377400953         T         A   
28735          2  232545769.0   rs962602943         G         T   
28736          2  232545774.0   rs970615017         A         G   
28737          2  232545783.0   rs980712487         G         A   

               Source  Filters - exomes Filters - genomes HGVS Consequence  \
0      gnomAD Genomes               NaN              PASS         c.*71C>T   
1      gnomAD Genomes               NaN